In [1]:
from pyspark.sql import SparkSession,DataFrame
import os
import numpy as np
from pyspark.sql import functions as f
from pyspark.sql.types import *
import io
import time
from pyspark.sql import Row
from pyspark.sql.types import StructType,StructField, StringType, FloatType

In [2]:
# worker_image="inseefrlab/jupyter-datascience:py3.9.7-spark3.1.2"
worker_image="inseefrlab/jupyter-datascience:master"

local=False
if local:
    spark = SparkSession \
    .builder.master("local[4]") \
    .appName("SparkReadLargeDataSet") \
    .getOrCreate()
else: 
    spark = SparkSession \
    .builder.master("k8s://https://kubernetes.default.svc:443") \
    .appName("SparkReadLargeDataSet") \
    .config("spark.kubernetes.container.image", worker_image) \
    .config("spark.kubernetes.authenticate.driver.serviceAccountName", os.environ['KUBERNETES_SERVICE_ACCOUNT']) \
    .config("spark.kubernetes.driver.pod.name", os.environ['KUBERNETES_POD_NAME']) \
    .config("spark.executor.instances", "32") \
    .config("spark.executor.memory","8g") \
    .config("spark.kubernetes.namespace", os.environ['KUBERNETES_NAMESPACE']) \
    .getOrCreate()

In [ ]:
spark.sparkContext.stop()

In [3]:
! kubectl get pods

I1108 16:31:41.027996    2877 request.go:655] Throttling request took 1.169132298s, request: GET:https://kubernetes.default/apis/wgpolicyk8s.io/v1alpha1?timeout=32s
NAME                                               READY   STATUS    RESTARTS   AGE
amundsen-elasticsearch-master-0                    1/1     Running   0          19d
amundsen-elasticsearch-master-1                    1/1     Running   0          19d
amundsen-elasticsearch-master-2                    1/1     Running   0          19d
amundsen-test-frontend-5648d65454-ffgpt            1/1     Running   0          19d
amundsen-test-metadata-c57c47666-7bjsz             1/1     Running   0          19d
amundsen-test-search-5c8455d784-g56vh              1/1     Running   0          19d
datahub-datahub-frontend-6ff5494f44-7gcpx          1/1     Running   0          19d
datahub-datahub-gms-5b66978446-g2526               1/1     Running   0          19d
elasticsearch-master-0                             1/1     Running   0         

In [5]:
data_path="s3a://pengfei/diffusion/data_format/ny_taxis/parquet/raw_2011_2012"

In [7]:
schema=StructType([ \
    StructField("vendor_id",StringType(),True), \
    StructField("pickup_at",StringType(),True), \
    StructField("dropoff_at",StringType(),True), \
    StructField("passenger_count", StringType(), True), \
    StructField("trip_distance", FloatType(), True), \
    StructField("pickup_longitude", FloatType(), True), \
    StructField("pickup_latitude", FloatType(), True), \
    StructField("rate_code_id", StringType(), True), \
    StructField("store_and_fwd_flag", StringType(), True), \
    StructField("dropoff_longitude", FloatType(), True), \
    StructField("dropoff_latitude", FloatType(), True), \
    StructField("payment_type", StringType(), True), \
    StructField("fare_amount", FloatType(), True), \
    StructField("extra",FloatType(), True),\
    StructField("mta_tax", FloatType(), True), \
    StructField("tip_amount",FloatType(), True),\
    StructField("tolls_amount", FloatType(), True), \
    StructField("total_amount",FloatType(), True),\
  ])

In [8]:
df=spark.read.schema(schema).parquet(data_path)
df.show()

Py4JJavaError: An error occurred while calling o81.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 1.0 failed 4 times, most recent failure: Lost task 0.3 in stage 1.0 (TID 38) (10.233.115.114 executor 17): java.lang.NullPointerException
	at org.apache.spark.sql.execution.vectorized.OnHeapColumnVector.putLong(OnHeapColumnVector.java:327)
	at org.apache.spark.sql.execution.datasources.parquet.VectorizedColumnReader.decodeDictionaryIds(VectorizedColumnReader.java:418)
	at org.apache.spark.sql.execution.datasources.parquet.VectorizedColumnReader.readBatch(VectorizedColumnReader.java:280)
	at org.apache.spark.sql.execution.datasources.parquet.VectorizedParquetRecordReader.nextBatch(VectorizedParquetRecordReader.java:283)
	at org.apache.spark.sql.execution.datasources.parquet.VectorizedParquetRecordReader.nextKeyValue(VectorizedParquetRecordReader.java:181)
	at org.apache.spark.sql.execution.datasources.RecordReaderIterator.hasNext(RecordReade

In [5]:
df.count()

1547741381

In [6]:
df.rdd.getNumPartitions()

312

In [7]:
df.printSchema()

root
 |-- vendor_id: string (nullable = true)
 |-- pickup_at: timestamp (nullable = true)
 |-- dropoff_at: timestamp (nullable = true)
 |-- passenger_count: byte (nullable = true)
 |-- trip_distance: float (nullable = true)
 |-- pickup_longitude: float (nullable = true)
 |-- pickup_latitude: float (nullable = true)
 |-- rate_code_id: integer (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- dropoff_longitude: float (nullable = true)
 |-- dropoff_latitude: float (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- fare_amount: float (nullable = true)
 |-- extra: float (nullable = true)
 |-- mta_tax: float (nullable = true)
 |-- tip_amount: float (nullable = true)
 |-- tolls_amount: float (nullable = true)
 |-- total_amount: float (nullable = true)



In [12]:
df2009=df.filter(f.year(f.col("pickup_at"))==2009)

In [16]:
output_path="s3a://projet-spark-lab/test_pengfei/2009"
df2009.write.parquet(output_path)

Py4JJavaError: An error occurred while calling o83.parquet.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:231)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:188)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:108)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:106)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:131)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$execute$1(SparkPlan.scala:180)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$executeQuery$1(SparkPlan.scala:218)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.sql.execution.SparkPlan.executeQuery(SparkPlan.scala:215)
	at org.apach